In [1]:
%load_ext autoreload
%autoreload 2
from metrics import CorefEvaluator
from document import Document
import json
import os

c:\users\tollef\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [2]:
from VISUALIZATION import highlighter as viz
from IPython.core.display import display, HTML
display(HTML(open('VISUALIZATION/highlighter/highlight.css').read()))
display(HTML(open('VISUALIZATION/highlighter/highlight.js').read()))

In [3]:
import spacy
nlp = spacy.load("en_core_web_md")
disabled = ["tagger", "parser", "ner", "textcat"]  # keep only tokenization

In [3]:
from MODEL_WRAPPERS.Corenlp import CoreNLP
corenlp = CoreNLP(ram="3G", viz=viz)

Starting server with command: java -Xmx3G -cp C:\Users\Tollef\Documents\GitHub\masterNEW\experiments\Coreference\Stanza (corenlp)\stanford-corenlp-4.0.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-a00b78b389d2499e.props -preload coref
Starting server with command: java -Xmx3G -cp C:\Users\Tollef\Documents\GitHub\masterNEW\experiments\Coreference\Stanza (corenlp)\stanford-corenlp-4.0.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-a00b78b389d2499e.props -preload coref


In [4]:
from MODEL_WRAPPERS.Neuralcoref import Coref
params = {
    "greed": 0.54,
    "max_dist": 100,
    "max_dist_match": 500
}
neuralcoref = Coref(params, spacy_size="md", viz=viz)

In [8]:
from MODEL_WRAPPERS.Spanbert import SpanBert
spanbert = SpanBert(viz=viz)

2020-05-14 12:03:48 WARNING: error loading _jsonnet (this is expected on Windows), treating ../../pretrained/allen-spanbert-large/config.json as plain json
2020-05-14 12:03:57 WARNING: Did not use initialization regex that was passed: _context_layer._module.weight_ih.*
2020-05-14 12:03:57 WARNING: Did not use initialization regex that was passed: _context_layer._module.weight_hh.*


In [9]:
models = [corenlp, neuralcoref, spanbert]

In [5]:
data_path = os.path.join(os.path.dirname(os.getcwd()), "coreference_data")

dev_path = os.path.join(data_path, "dev_data")

datasets = [os.path.join(dev_path, f) for f in os.listdir(dev_path)]
datasets

['C:\\Users\\Tollef\\Documents\\GitHub\\masterNEW\\REPO\\coreference_data\\dev_data\\gum.coreflite',
 'C:\\Users\\Tollef\\Documents\\GitHub\\masterNEW\\REPO\\coreference_data\\dev_data\\litbank.coreflite',
 'C:\\Users\\Tollef\\Documents\\GitHub\\masterNEW\\REPO\\coreference_data\\dev_data\\ontonotes_dev.coreflite',
 'C:\\Users\\Tollef\\Documents\\GitHub\\masterNEW\\REPO\\coreference_data\\dev_data\\preco_dev.coreflite']

In [6]:
tokens = ["''", "Is", "there", "anything", "else", "you", "need", ",", "honey", "?", "''", "my", "dad", "asked", "me", "as", "he", "put", "three", "twenty", "dollar", "bills", "in", "my", "hand", ".", "I", "was", "traveling", "back", "home", "from", "a", "family", "visit", ",", "and", "after", "treating", "me", "to", "breakfast", "and", "filling", "my", "car", "with", "gas", ",", "it", "was", "obvious", "that", "my", "dad", "wanted", "to", "make", "sure", "that", "I", "would", "be", "okay", "on", "the", "road", ".", "''", "No", ",", "Dad", ".", "You", "'ve", "done", "so", "much", "already", ".", "Thank", "you", "!", "''", "I", "was", "overwhelmed", "once", "again", "by", "his", "kind", "acts", "of", "providing", "everything", "I", "needed", ",", "although", "I", "turned", "40", ".", "Yet", "I", "realize", "that", "in", "my", "father", "'s", "eyes", ",", "I", "will", "always", "be", "his", "little", "girl", ".", "He", "takes", "deep", "pleasure", "in", "knowing", "his", "children", "are", "all", "right", ".", "Now", "that", "he", "has", "enough", "money", ",", "he", "loves", "to", "give", "whenever", "he", "sees", "a", "need", ".", "But", "this", "was", "not", "always", "the", "case", ".", "Divorced", "from", "my", "mother", "when", "I", "was", "11", ",", "my", "dad", "could", "n't", "be", "around", "his", "kids", "as", "often", "as", "he", "would", "have", "liked", ".", "Money", "was", "also", "tight", ";", "even", "weekend", "visits", "were", "rare", ".", "However", ",", "my", "dad", "stayed", "in", "constant", "communication", "with", "us", "and", "made", "sure", "he", "was", "involved", "in", "our", "lives", ".", "Though", "he", "could", "n't", "always", "be", "there", "in", "person", ",", "I", "knew", "he", "was", "only", "a", "phone", "call", "away", ".", "I", "could", "always", "make", "sure", "of", "that", ".", "Even", "now", ",", "almost", "30", "years", "later", ",", "I", "treasure", "knowing", "that", "I", "can", "pick", "up", "the", "phone", "and", "call", "Dad", ",", "and", "he", "'ll", "be", "there", "for", "me", ".", "I", "have", "a", "wonderful", "husband", ",", "but", "that", "has", "n't", "changed", "how", "Dad", "sees", "me", ".", "I", "'m", "still", "his", "child", "and", "he", "loves", "to", "see", "that", "my", "needs", "are", "met", ".", "I", "remember", "a", "time", "when", "I", "was", "shopping", "in", "a", "hardware", "store", "with", "Dad", ".", "I", "mentioned", "my", "plans", "to", "paint", "one", "wall", "in", "my", "house", ".", "Well", ",", "that", "'s", "all", "it", "took", "for", "Dad", "to", "take", "action", ".", "By", "the", "time", "I", "got", "to", "the", "checkout", "line", ",", "all", "the", "supplies", "I", "picked", "out", "were", "put", "out", "of", "my", "hands", "and", "placed", "with", "things", "he", "bought", ".", "Then", "there", "was", "the", "time", "when", "I", "took", "him", "with", "me", "to", "do", "some", "grocery", "shopping", "for", "just", "a", "few", "''", "items", "''", ".", "By", "the", "time", "we", "were", "finished", ",", "my", "shopping", "cart", "was", "full", "of", "groceries", "from", "every", "shelf", "in", "the", "store", "!", "My", "sister", "and", "I", "joke", "that", "if", "you", "do", "n't", "want", "Dad", "to", "buy", "it", "for", "you", ",", "avoid", "even", "mentioning", "you", "want", "something", "."]
text = ' '.join(tokens)
text

"'' Is there anything else you need , honey ? '' my dad asked me as he put three twenty dollar bills in my hand . I was traveling back home from a family visit , and after treating me to breakfast and filling my car with gas , it was obvious that my dad wanted to make sure that I would be okay on the road . '' No , Dad . You 've done so much already . Thank you ! '' I was overwhelmed once again by his kind acts of providing everything I needed , although I turned 40 . Yet I realize that in my father 's eyes , I will always be his little girl . He takes deep pleasure in knowing his children are all right . Now that he has enough money , he loves to give whenever he sees a need . But this was not always the case . Divorced from my mother when I was 11 , my dad could n't be around his kids as often as he would have liked . Money was also tight ; even weekend visits were rare . However , my dad stayed in constant communication with us and made sure he was involved in our lives . Though he 

In [7]:
precoclusters = [[[3, 6]], [[18, 21]], [[19, 20]], [[23, 24]], [[30, 30]], [[32, 34]], [[33, 33]], [[41, 41]], [[44, 45]], [[47, 47]], [[65, 66]], [[90, 97]], [[95, 97]], [[102, 102]], [[109, 112]], [[124, 125]], [[128, 129], [174, 175], [204, 204], [212, 212]], [[134, 134]], [[138, 139]], [[148, 149]], [[152, 152]], [[156, 157]], [[161, 162]], [[166, 166]], [[184, 184]], [[190, 191]], [[190, 190]], [[212, 213]], [[223, 223]], [[230, 232]], [[241, 241]], [[245, 245]], [[259, 260]], [[275, 277]], [[280, 280]], [[284, 287]], [[300, 301]], [[314, 316]], [[315, 315]], [[322, 323]], [[326, 330]], [[329, 330]], [[334, 334]], [[343, 343]], [[351, 353]], [[352, 352]], [[355, 360]], [[365, 366]], [[370, 372]], [[375, 375]], [[384, 384], [380, 380], [310, 310], [161, 161], [164, 164], [168, 168], [5, 5], [11, 11], [14, 14], [26, 26], [44, 44], [53, 53], [23, 23], [39, 39], [60, 60], [8, 8], [84, 84], [105, 105], [109, 109], [118, 120], [100, 100], [96, 96], [225, 225], [235, 235], [273, 273], [289, 289], [271, 271], [251, 251], [300, 300], [255, 255], [287, 287], [320, 320], [306, 306], [322, 322], [348, 348], [329, 329], [358, 358], [365, 365], [419, 419], [422, 422], [405, 405], [114, 114], [197, 197], [292, 293]], [[387, 389]], [[388, 388]], [[399, 400]], [[401, 401]], [[405, 407]], [[413, 417]], [[416, 417]], [[419, 420]], [[419, 422]], [[430, 430], [11, 12], [16, 16], [53, 54], [71, 71], [73, 73], [81, 81], [90, 90], [109, 111], [118, 118], [122, 122], [136, 136], [141, 141], [128, 128], [146, 146], [168, 169], [174, 174], [179, 179], [197, 198], [216, 216], [227, 227], [208, 208], [295, 295], [292, 292], [263, 263], [266, 266], [285, 285], [318, 318], [340, 340], [371, 371], [382, 382]], [[439, 442]], [[440, 440], [426, 426], [435, 435]], [[442, 442], [433, 433], [411, 417]], [[94, 97]], [[201, 202]], [[247, 249]], [[247, 248]], [[308, 318]], [[346, 353]], [[378, 396]], [[392, 396]], [[38, 47]], [[127, 132]], [[231, 231]], [[43, 47]], [[38, 41]]]
clusters = []
for cl in precoclusters:
    if len(cl) > 1:
        clusters.append(cl)

In [8]:
# print out clusters
def print_clusters(tkns, cl):
    for c in cl:
        print("New cluster")
        for m in c:
            m1, m2 = m
            print("({}):".format(m), tkns[m1:m2+1])

In [9]:
viz.raw_render(tokens, clusters)

In [12]:
model = neuralcoref
print(model)
pred = model.predict(text)
print(pred)
print("token length", len(model.get_tokens()))
print("true token length", len(tokens))
viz.raw_render(tokens, pred)

[[[11, 12], [16, 16], [53, 54], [71, 71], [91, 91], [119, 119], [123, 123], [129, 129], [137, 137], [142, 142], [147, 147], [169, 170], [175, 175], [180, 180], [198, 199], [209, 209], [217, 217], [228, 228], [264, 264], [267, 267], [272, 272], [286, 286], [294, 294], [297, 297], [320, 320], [342, 342], [373, 373], [384, 384], [386, 386], [432, 432], [437, 437]], [[205, 205], [213, 213], [403, 403]], [[276, 278], [293, 295]], [[316, 320], [418, 419]], [[324, 332], [339, 339]], [[407, 409], [435, 435]]]
token length 446
true token length 444


In [13]:
viz.raw_render(model.get_tokens(), pred)

In [160]:
# hadle incorrect tokenization:
# iterate the ORIGINAL INPUT tokens
model_tokens = model.get_tokens()
modcount = 0
for index, tok in enumerate(tokens):
    if tok != model_tokens[index]:
        # mod count
        modcount += 1
        #print(tok, "!=", spacytokens[index])
        # the original document has been retokenized, update all links above this index
        newclust = []
        for cluster in pred:
            mentionclust = []
            for mention in cluster:
                m1, m2 = mention
                if m1 > index:
                    #print("Token m1", m1, model_tokens[m1])
                    m1 -= 1
                if m2 > index:
                    #print("Token m2", m1, model_tokens[m2])
                    m2 -= 1
                mentionclust.append([m1, m2])
            newclust.append(mentionclust)
        pred = newclust
                
print("modded", modcount)
#print("Comparing tokens | model tokens")
#for x, y in zip(tokens, model.get_tokens()):
#    print(x, "\t ", y)
#print("reduced")
#viz.raw_render(tokens, cluster)

modded 369


In [34]:
def get_offset_indexes(original_tokens, model_tokens):
    idx = 0
    model_idx = 0
    
    offset_indexes = set()

    while idx < len(original_tokens):
        tok = original_tokens[idx]
        modtok = model_tokens[model_idx]
        if tok != modtok:
            offset_indexes.add(idx)
            model_idx += 1
        idx += 1
        model_idx += 1
    
    return list(offset_indexes)

In [39]:
def alter_index(index):
    for c_idx in range(len(pred)):
        for m_idx in range(len(pred[c_idx])):
            m1, m2 = pred[c_idx][m_idx]
            if m1 > index:
                m1 -= 1
            if m2 > index:
                m2 -= 1
            pred[c_idx][m_idx] = [m1, m2]

In [40]:
for offset_index in get_offset_indexes(tokens, model.get_tokens()):
    alter_index(offset_index)

In [42]:
viz.raw_render(tokens, pred)

# Resolving issues
The original preco files contain lots of unicode quotation marks, as opposed to the standardized "". This results in difficulties parsing, and the original dataset needs to be transformed, as well as updating the mention indices.

In [36]:
from unidecode import unidecode

In [75]:
weird = set()
gum, lit, onto, prec = datasets

dataset=prec

with open(dataset, "r", encoding="utf8") as data:
    all_docs = data.readlines()
    for doc in all_docs:
        doc = json.loads(doc)
        tkns = doc["tokens"]
        for tok in tkns:
            tok = unidecode(tok)
            if len(tok) == 1 and not tok.isalnum():
                weird.add(tok)

In [76]:
weird

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 ']',
 '^',
 '_',
 '~'}

# Lemmas and shit

In [51]:
# retokenize the pre-tokenized input
joined = ' '.join(tokens)
doc = nlp(joined, disable=disabled)
doc

'' Is there anything else you need , honey ? '' my dad asked me as he put three twenty dollar bills in my hand . I was traveling back home from a family visit , and after treating me to breakfast and filling my car with gas , it was obvious that my dad wanted to make sure that I would be okay on the road . '' No , Dad . You 've done so much already . Thank you ! '' I was overwhelmed once again by his kind acts of providing everything I needed , although I turned 40 . Yet I realize that in my father 's eyes , I will always be his little girl . He takes deep pleasure in knowing his children are all right . Now that he has enough money , he loves to give whenever he sees a need . But this was not always the case . Divorced from my mother when I was 11 , my dad could n't be around his kids as often as he would have liked . Money was also tight ; even weekend visits were rare . However , my dad stayed in constant communication with us and made sure he was involved in our lives . Though he c

In [54]:
spacytokens = [t.text for t in doc]
for sp, norm in zip(spacytokens, tokens):
    print(sp, "\t", norm)

'' 	 ''
Is 	 Is
there 	 there
anything 	 anything
else 	 else
you 	 you
need 	 need
, 	 ,
honey 	 honey
? 	 ?
'' 	 ''
my 	 my
dad 	 dad
asked 	 asked
me 	 me
as 	 as
he 	 he
put 	 put
three 	 three
twenty 	 twenty
dollar 	 dollar
bills 	 bills
in 	 in
my 	 my
hand 	 hand
. 	 .
I 	 I
was 	 was
traveling 	 traveling
back 	 back
home 	 home
from 	 from
a 	 a
family 	 family
visit 	 visit
, 	 ,
and 	 and
after 	 after
treating 	 treating
me 	 me
to 	 to
breakfast 	 breakfast
and 	 and
filling 	 filling
my 	 my
car 	 car
with 	 with
gas 	 gas
, 	 ,
it 	 it
was 	 was
obvious 	 obvious
that 	 that
my 	 my
dad 	 dad
wanted 	 wanted
to 	 to
make 	 make
sure 	 sure
that 	 that
I 	 I
would 	 would
be 	 be
okay 	 okay
on 	 on
the 	 the
road 	 road
. 	 .
'' 	 ''
No 	 No
, 	 ,
Dad 	 Dad
. 	 .
You 	 You
' 	 've
ve 	 done
done 	 so
so 	 much
much 	 already
already 	 .
. 	 Thank
Thank 	 you
you 	 !
! 	 ''
'' 	 I
I 	 was
was 	 overwhelmed
overwhelmed 	 once
once 	 again
again 	 by
by 	 his
his 	 kind
ki

In [28]:
lemmed_spacy = [t if t.lemma_ == "-PRON-" else t.lemma_ for t in doc]
lemmed_norm = [t if t.lemma_ == "-PRON-" else t.lemma_ for t in doc]
for orig, lem in zip(doc, lemmed):
    print(orig.text, "\t", lem)

'' 	 ''
Is 	 be
there 	 there
anything 	 anything
else 	 else
you 	 you
need 	 need
, 	 ,
honey 	 honey
? 	 ?
'' 	 ''
my 	 my
dad 	 dad
asked 	 ask
me 	 me
as 	 as
he 	 he
put 	 put
three 	 three
twenty 	 twenty
dollar 	 dollar
bills 	 bill
in 	 in
my 	 my
hand 	 hand
. 	 .
I 	 I
was 	 be
traveling 	 travel
back 	 back
home 	 home
from 	 from
a 	 a
family 	 family
visit 	 visit
, 	 ,
and 	 and
after 	 after
treating 	 treat
me 	 me
to 	 to
breakfast 	 breakfast
and 	 and
filling 	 fill
my 	 my
car 	 car
with 	 with
gas 	 gas
, 	 ,
it 	 it
was 	 be
obvious 	 obvious
that 	 that
my 	 my
dad 	 dad
wanted 	 want
to 	 to
make 	 make
sure 	 sure
that 	 that
I 	 I
would 	 would
be 	 be
okay 	 okay
on 	 on
the 	 the
road 	 road
. 	 .
'' 	 ''
No 	 no
, 	 ,
Dad 	 Dad
. 	 .
You 	 You
' 	 '
ve 	 ve
done 	 do
so 	 so
much 	 much
already 	 already
. 	 .
Thank 	 thank
you 	 you
! 	 !
'' 	 ''
I 	 I
was 	 be
overwhelmed 	 overwhelm
once 	 once
again 	 again
by 	 by
his 	 his
kind 	 kind
acts 	 act
of 	

In [102]:
_tokens = tokens.copy()
_clusters = clusters.copy()
DISABLED = ["tagger", "parser", "ner", "textcat"]

In [104]:
def alter_cluster_mentions_by_index(index, direction):
    for c_idx in range(len(_clusters)):
        for m_idx in range(len(_clusters[c_idx])):
            try:
                m1, m2 = _clusters[c_idx][m_idx]
            except Exception:
                print(_clusters[c_idx][m_idx])
            if m1 > index:
                m1 += direction * 1
            if m2 > index:
                m2 += direction * 1
            _clusters[c_idx][m_idx] = [m1, m2]

def remove_invalid_cluster_tokens():
    # start by creating a spacy tokenization
    spacydoc = nlp(' '.join(_tokens), disable=DISABLED)
    spacytoks = [t.text for t in spacydoc]

    latex_quotations = ["``"]
    parsed_tokens = []
    
    for index, token in enumerate(_tokens):
        if token == " ":
            alter_cluster_mentions_by_index(index, -1)

        else:
            # fix latex quotations.
            if token in latex_quotations:
                token = "''"
            elif token != spacytoks[index]:
                # e.g. 've becomes ' and ve, as separate tokens.
                # fix this by expanding index
                #alter_cluster_mentions_by_index(index, 1)
                pass
            parsed_tokens.append(token)
    print(parsed_tokens)

In [105]:
remove_invalid_cluster_tokens()

["''", 'Is', 'there', 'anything', 'else', 'you', 'need', ',', 'honey', '?', "''", 'my', 'dad', 'asked', 'me', 'as', 'he', 'put', 'three', 'twenty', 'dollar', 'bills', 'in', 'my', 'hand', '.', 'I', 'was', 'traveling', 'back', 'home', 'from', 'a', 'family', 'visit', ',', 'and', 'after', 'treating', 'me', 'to', 'breakfast', 'and', 'filling', 'my', 'car', 'with', 'gas', ',', 'it', 'was', 'obvious', 'that', 'my', 'dad', 'wanted', 'to', 'make', 'sure', 'that', 'I', 'would', 'be', 'okay', 'on', 'the', 'road', '.', "''", 'No', ',', 'Dad', '.', 'You', "'ve", 'done', 'so', 'much', 'already', '.', 'Thank', 'you', '!', "''", 'I', 'was', 'overwhelmed', 'once', 'again', 'by', 'his', 'kind', 'acts', 'of', 'providing', 'everything', 'I', 'needed', ',', 'although', 'I', 'turned', '40', '.', 'Yet', 'I', 'realize', 'that', 'in', 'my', 'father', "'s", 'eyes', ',', 'I', 'will', 'always', 'be', 'his', 'little', 'girl', '.', 'He', 'takes', 'deep', 'pleasure', 'in', 'knowing', 'his', 'children', 'are', 'all',

In [106]:
viz.raw_render(_tokens, _clusters)

In [101]:
print_clusters(tokens, clusters)

New cluster
([128, 129]): ['his', 'children']
([174, 175]): ['his', 'kids']
([204, 204]): ['us']
([212, 212]): ['our']
New cluster
([384, 384]): ['me']
([380, 380]): ['I']
([310, 310]): ['I']
([161, 161]): ['my']
([164, 164]): ['I']
([168, 168]): ['my']
([5, 5]): ['you']
([11, 11]): ['my']
([14, 14]): ['me']
([26, 26]): ['I']
([44, 44]): ['my']
([53, 53]): ['my']
([23, 23]): ['my']
([39, 39]): ['me']
([60, 60]): ['I']
([8, 8]): ['honey']
([84, 84]): ['I']
([105, 105]): ['I']
([109, 109]): ['my']
([118, 120]): ['his', 'little', 'girl']
([100, 100]): ['I']
([96, 96]): ['I']
([225, 225]): ['I']
([235, 235]): ['I']
([273, 273]): ['I']
([289, 289]): ['I']
([271, 271]): ['me']
([251, 251]): ['I']
([300, 300]): ['my']
([255, 255]): ['I']
([287, 287]): ['me']
([320, 320]): ['I']
([306, 306]): ['remember']
([322, 322]): ['my']
([348, 348]): ['I']
([329, 329]): ['my']
([358, 358]): ['I']
([365, 365]): ['my']
([419, 419]): ['My']
([422, 422]): ['I']
([405, 405]): ['my']
([114, 114]): ['I']
([197,

In [100]:
viz.raw_render(tokens, clusters)